First, we set up the MNIST dataset and load everything we need.

In [1]:
BATCH_SIZE = 16
WEIGHTS_INITIALIZER = tf.contrib.layers.xavier_initializer()

# image_height = image_width = 28, num_channels = 1
dataset, image_height, image_width, num_channels, next_train_batch, next_test_batch = load_dataset('mnist')
train_data = dataset.train
test_data = dataset.test



NameError: name 'load_dataset' is not defined

Next we define placeholders for the input to our network and a utility function to get the shape of a tensor:

In [2]:
inputs = tf.placeholder(tf.float32, [None, image_height, image_width, 1])

def get_shape(tensor):
    return tensor.get_shape().as_list()

NameError: name 'tf' is not defined

Next, we create a function to perform 2-D image convolutions with the masking procedure outlined above. We also create a 1-D convolution for applying the 1-D kernels to the skewed image in the Diagonal BiLSTM.

In [3]:
def conv2d(inputs, num_outputs, kernel_shape, mask_type='A'):
    batch_size, image_height, image_width, num_channels = get_shape(inputs)

    kernel_height, kernel_width = kernel_shape
    # get the location of the pixel being predicted by this kernel
    center_height, center_width = kernel_height // 2, kernel_width // 2
    
    # initialize kernel weights
    weights_shape = [kernel_height, kernel_width, 1, num_outputs]
    weights = tf.get_variable('weights', weights_shape, tf.float32, WEIGHTS_INITIALIZER)
    
    # create and apply the masks to the convolution
    # set all pixels below and all to the right of the center to 0
    mask = np.ones([kernel_height, kernel_width, 1, num_outputs], dtype=np.float32)
    mask[center_height, center_width+1:, :, :] = 0.
    mask[center_height+1:, :, :, :] = 0.
    
    # for type A masks, we do not allow self-connections
    if mask_type == 'A':
        mask[center_height, center_width, :, :] = 0.
        
    # apply the mask
    weights *= tf.constant(mask, dtype=tf.float32)
    
    # apply the convolution
    outputs = tf.nn.conv2d(inputs, weights, [1, 1, 1, 1], padding='SAME')
    return outputs

def conv1d(inputs, num_outputs, kernel_height):
    batch_size, image_height, image_width, num_channels = get_shape(inputs)
    kernel_height, kernel_width = kernel_height, 1
    
    weights_shape = [kernel_height, kernel_width, 1, num_outputs]
    weights = tf.get_variable('weights', weights_shape, tf.float32, WEIGHT_INITIALIZER)
    outputs = tf.nn.conv2d(inputs, weights, [1, 1, 1, 1], padding='SAME')
    return outputs
